## [Day 17](https://adventofcode.com/2020/day17)

Here we have another geometric style puzzle where we have a 3D space full of nodes. At each node, we can have the state on (#) or off (.). At each cyle, the nodes consider the 1 unit cube (26 values) around them.. The rules about changing each node are:
1. If the node is active and 2 or 3 of their neighbors are active, it stays active. Otherwise it goes inactive.
2. If the node is inactive and three of their neighbors are active, it becomes active. Otherwise, it becomes inactive.

We're asked to run this process 6 times on the initial input and then count the number of active nodes.

The challange for this seems that the space is infinite so we're going to come up with a way of not going 'out of bounds' on our indices.

In [1]:
import pandas as pd
import numpy as np
import itertools
nodes = open('../inputs/d17.txt').read().splitlines()
nodes

So I think this will be easiest to store as a dictionary with triplets (tuples) as the keys and the active status as the value. Let's initialize our dictionary with the puzzle input. I don't really care that much about maintaining the visual aspect of this (since I'm bad at that kind of thing anyways) so let's just have downward along the list be y increasing.

In [2]:
space = {}
for y, row in enumerate(nodes):
    for x, value in enumerate(row):
        space.update({(x,y,0):(value)})

In [3]:
space

Now how about a function to add a 'skin' to the space. I need to be able to expand the space when we need more room to place values. We could do this just from knowing the initial dimensions but I kind of want to program this in a way that doesn't care about magic numbers.

In [4]:
def add_skin(cycle):
    # Get the mins and maxes of each dimension:
    xs = [key[0] for key in cycle.keys()]
    ys = [key[1] for key in cycle.keys()]
    zs = [key[2] for key in cycle.keys()]
    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)
    zmin, zmax = min(zs), max(zs)
    # Then the range of value
    xrange = list(range(xmin-1, xmax+2))
    yrange = list(range(ymin-1, ymax+2))
    zrange = list(range(zmin-1, zmax+2))
    # now we just loop through. Adding the tuples is kind of interesting
    # If you write something like (x,) then python knows this is a 1-tuple
    for tupe in itertools.product(xrange, yrange):
        cycle.update({tupe+(zmin-1,) : '.', tupe+(zmax+1,) : '.'})
    for tupe in itertools.product(xrange, zrange):
        cycle.update({tupe[:1]+(ymin-1,)+tupe[-1:] : '.', tupe[:1]+(ymax+1,)+tupe[-1:] : '.'})    
    for tupe in itertools.product(yrange, zrange):
        cycle.update({(xmin-1,)+tupe : '.', (xmax+1,)+tupe : '.'})
    return cycle

Now a function to get the neighbors of a given location:

In [5]:
def get_neighbs(tupe):
    x, y, z = tupe[0], tupe[1], tupe[2]
    neighbors = list(itertools.product([x-1, x, x+1], [y-1, y, y+1], [z-1, z, z+1]))
    neighbors = [tuple(x) for x in neighbors]
    neighbors.remove(tupe)
    tot = 0
    for neighbor in neighbors:
        if neighbor in space:
            tot += space[neighbor] == '#'
    return tot

In [6]:
space[(0, 1, 0)]

Finally, we run through some iterations:

In [7]:
for i in range(6):
    # Add a new skin outwards
    space = add_skin(space)
    # make a copy to store the next round in:
    next_space = space.copy()
    # Iterate through the last dictionary and save to the new:
    for loc in space:
        neighbs = get_neighbs(loc)
        if space[loc] == '#':
            if neighbs not in [2,3]:
                next_space.update({loc: '.'})
        else:
            if neighbs == 3:
                next_space.update({loc:'#'})
    # reset
    space = next_space


In [8]:
sol1 = 0
for loc in space:
    if space[loc] == '#':
        sol1 += 1
sol1        

375

### Part 2

So for part 2 they basically ask us to repeat this process with another dimension added. I don't think this will be that conceptually difficult but I have to go up and recopy the whole thing with some extra diddly whoos. Makes me wonder how I could have done this without the amount of repitition below:

In [9]:
space2 = {}
for y, row in enumerate(nodes):
    for x, value in enumerate(row):
        space2.update({(x,y,0,0):(value)})
        
def add_skin2(cycle):
    # Get the mins and maxes of each dimension:
    xs = [key[0] for key in cycle.keys()]
    ys = [key[1] for key in cycle.keys()]
    zs = [key[2] for key in cycle.keys()]
    rs = [key[3] for key in cycle.keys()]
    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)
    zmin, zmax = min(zs), max(zs)
    rmin, rmax = min(rs), max(rs)
    # Then the range of value
    xrange = list(range(xmin-1, xmax+2))
    yrange = list(range(ymin-1, ymax+2))
    zrange = list(range(zmin-1, zmax+2))
    rrange = list(range(rmin-1, rmax+2))
    # now we just loop through. Adding the tuples is kind of interesting
    # If you write something like (x,) then python knows this is a 1-tuple
    for tupe in itertools.product(xrange, yrange, rrange):
        cycle.update({tupe[:2]+(zmin-1,) + tupe[-1:] : '.', tupe[:2]+(zmax+1,)+tupe[-1:] : '.'})
    for tupe in itertools.product(xrange, zrange, rrange):
        cycle.update({tupe[:1]+(ymin-1,)+tupe[-2:] : '.', tupe[:1]+(ymax+1,)+tupe[-2:] : '.'})    
    for tupe in itertools.product(yrange, zrange, rrange):
        cycle.update({(xmin-1,)+tupe : '.', (xmax+1,)+tupe : '.'})
    for tupe in itertools.product(xrange, yrange, zrange):
        cycle.update({tupe + (rmin-1,) : '.', tupe+(rmax+1,) : '.'})    
    return cycle        
                      
def get_neighbs2(tupe):
    x, y, z, r = tupe[0], tupe[1], tupe[2], tupe[3]
    neighbors = list(itertools.product([x-1, x, x+1], [y-1, y, y+1], [z-1, z, z+1], [r-1, r, r+1]))
    neighbors = [tuple(x) for x in neighbors]
    neighbors.remove(tupe)
    tot = 0
    for neighbor in neighbors:
        if neighbor in space2:
            tot += space2[neighbor] == '#'
    return tot                      

In [10]:
for i in range(6):
    # Add a new skin outwards
    space2 = add_skin2(space2)
    # make a copy to store the next round in:
    next_space2 = space2.copy()
    # Iterate through the last dictionary and save to the new:
    for loc in space2:
        neighbs = get_neighbs2(loc)
        if space2[loc] == '#':
            if neighbs not in [2,3]:
                next_space2.update({loc: '.'})
        else:
            if neighbs == 3:
                next_space2.update({loc:'#'})
    # reset
    space2 = next_space2

sol2 = 0
for loc in space2:
    if space2[loc] == '#':
        sol2 += 1
sol2                       

2192

I'm quite surprised that this turned out to be one of the easiest ones for me. I'm 2/3 done with this year and spacial problems end up messing me up a lot more but I finished this entire day in less than an hour.